# Movie Data Analysis

## Overview

## Problem Statement

Data cleaning 

In [1]:
# Step 1: Import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Reading the data 

In [2]:
# Box Office Mojo 
bom_movie_gross = pd.read_csv('zippedData/bom.movie_gross.csv.gz')

# === The Numbers ===
tn_movie_budgets = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')

# === The Movie Database (TMDb) ===
tmdb_movies = pd.read_csv('zippedData/tmdb.movies.csv.gz')

# === Rotten Tomatoes ===
# === Rotten Tomatoes ===
rt_movies = pd.read_csv('zippedData/rt.movie_info.tsv.gz', sep='\t', encoding='latin-1')
rt_reviews = pd.read_csv('zippedData/rt.reviews.tsv.gz', sep='\t', encoding='latin-1')


Im.db.zip is basically a compressed folder with several .tsv IMDb files inside

In [3]:
import zipfile, pandas as pd

with zipfile.ZipFile('zippedData/im.db.zip') as z:
    print(z.namelist())   #  shows you all files inside



['im.db']


In [4]:
import zipfile

with zipfile.ZipFile("zippedData/im.db.zip", "r") as z:
    z.extractall("zippedData/")  # this will create 'zippedData/im.db'


The file contains  a single SQLite database File called im.db,meaning you need to open it as a SQLite database

In [5]:
import sqlite3
#import pandas as pd

conn = sqlite3.connect("zippedData/im.db")
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)



            name
0   movie_basics
1      directors
2      known_for
3     movie_akas
4  movie_ratings
5        persons
6     principals
7        writers


Now loading those tables into pandas DataFrames with simple SQL queriees

In [53]:
movie_basics = pd.read_sql("SELECT * FROM movie_basics;", conn)

directors = pd.read_sql("SELECT * FROM directors;", conn)
known_for = pd.read_sql("SELECT * FROM known_for;", conn)
movie_akas = pd.read_sql("SELECT * FROM movie_akas;", conn)
movie_ratings = pd.read_sql("SELECT * FROM movie_ratings;", conn)
persons = pd.read_sql("SELECT * FROM persons;", conn)
principals = pd.read_sql("SELECT * FROM principals;", conn)
writers = pd.read_sql("SELECT * FROM writers;", conn)

# IMDb dataset

Strengths:

Very detailed on movies’ metadata (runtime, genres, release years, countries).
Has personnel links (directors, actors, writers) via tconst and nconst.
Richest in size and coverage.

Weakness:

Does not contain financial data (budget, revenue, box office gross).
You only get content-related info, not business performance.

# Financial datasets

Box Office Mojo (BOM) → has domestic/international gross.

The Numbers → has budget + box office revenue.

The Movie DB (TMDB) → includes popularity, user ratings, sometimes budget/revenue.

# Why merging matters

IMDb gives what the movie is + who made it.

BOM, The Numbers, and TMDB give how much money it made / cost.

Together, you can analyze relationships like:

Do longer runtimes correlate with higher/lower revenue?

Which genres are most profitable?

Do certain directors/writers consistently make successful films?

That’s why your statement is true: “IMDb is the most detailed dataset for descriptive info, but it lacks financials. To evaluate movie success, it must be merged with financial datasets (The Numbers, Box Office Mojo, TMDB).”

 # Data Cleaning 
We’ll clean only the datasets that are most useful for analysis (IMDb + financials). Rotten Tomatoes/TMDB can be optional later.


# Datasets to Clean First

 1 IMDb tables (content & metadata)

movie_basics → (title, year, runtime, genres)

movie_ratings → (average rating, votes)

2 Box Office Mojo (bom_movie_gross)

Domestic & foreign gross

3 The Numbers (tn_movie_budgets)

Budget + gross

In [18]:
 movie_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         5 non-null      object 
 1   primary_title    5 non-null      object 
 2   original_title   5 non-null      object 
 3   start_year       5 non-null      int64  
 4   runtime_minutes  4 non-null      float64
 5   genres           5 non-null      object 
dtypes: float64(1), int64(1), object(4)
memory usage: 368.0+ bytes


In [8]:
# Check duplicates
movie_basics.duplicated().sum()

0

In [9]:
# Convert datatypes

movie_basics = pd.read_sql("SELECT * FROM movie_basics;", conn).head()
movie_basics


,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


# Step 1

Extract only the columns that we need 

In [10]:
runtime_df = movie_basics[['primary_title', 'start_year', 'runtime_minutes']]

# Why 

movie_basics has many columns (genres, tconst, etc.), but for runtime analysis we only care about:

primary_title → movie name (for identification & merging later)

start_year → release year (to filter by time & merge with financial datasets)

runtime_minutes → our main feature of interest (movie length)


# Step 2 : 

Remove movies that haven't been released yet 

In [11]:
runtime_df = runtime_df[runtime_df['start_year'] < 2025]

 # Why?

Some rows have future release years (e.g., 2023, 2025).

Since we only analyze historical performance, those rows would give misleading results.

Keeps dataset consistent with financial data (which only has past films).

# Step 3

Drop row with missing runtimes 

In [12]:
runtime_df = runtime_df.dropna(axis=0, subset=['runtime_minutes'])

# Why?

Missing runtimes = useless for analysis.

Dropping them ensures we don’t get NaN values messing up plots/stats.

# Step 4
Inspect the cleaned result

In [13]:
print(runtime_df.shape)     # how many rows/columns after cleaning
print(runtime_df.isna().sum())  # check if any nulls remain
runtime_df.head()           # preview first 5 rows
runtime_df.info()           # check datatypes
runtime_df.describe()       # quick stats (mean, min, max runtime)


(4, 3)
primary_title      0
start_year         0
runtime_minutes    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 0 to 4
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   primary_title    4 non-null      object 
 1   start_year       4 non-null      int64  
 2   runtime_minutes  4 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 128.0+ bytes


,start_year,runtime_minutes
count,4.000000,4.000000
mean,2016.750000,122.750000
std,2.629956,39.305428
min,2013.000000,80.000000
25%,2016.000000,105.500000
50%,2017.500000,118.000000
75%,2018.250000,135.250000
max,2019.000000,175.000000


# Why?

shape → see how much data we have left after cleaning.

isna() → make sure runtimes are fully clean.

head() → sanity check if columns look correct.

info() → confirm datatypes (start_year should be int, runtime_minutes int/float).

describe() → see runtime distribution (are there very short/long outliers?).

# Now we’re prepping The Numbers and TMDb release dates so they can align with IMDb’s start_year

In [15]:
tn_movie_budgets = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
tn_movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [16]:
tn_movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [20]:
#  The Movie Database (TMDb) 
tmdb_movies = pd.read_csv('zippedData/tmdb.movies.csv.gz')
tmdb_movies.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [21]:
tmdb_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


# Step 1
Convert release_date into datetime 

In [22]:
tn_movie_budgets['release_date'] = pd.to_datetime(tn_movie_budgets['release_date'])
tmdb_movies['release_date'] = pd.to_datetime(tmdb_movies['release_date'])


# Why?

Dates are often read in as strings → can’t extract year/month directly.

pd.to_datetime() standardizes them into true datetime objects.

 # step 2 
 
 Extract release year (to match IMDb format)

In [23]:
tn_movie_budgets['release_year'] = tn_movie_budgets['release_date'].dt.year
tmdb_movies['release_year'] = tmdb_movies['release_date'].dt.year


# Why?

IMDb uses just the year (start_year).

To merge datasets later, we need the same format (year only).

# Step 3: 
Extract release month (both numeric & string)

In [24]:
tn_movie_budgets['month_dt'] = tn_movie_budgets['release_date'].dt.month  # numeric month (1–12)
tn_movie_budgets['month'] = tn_movie_budgets['release_date'].dt.month     # duplicate here, can adjust if you want month names


# Why?

Month helps analyze seasonality (e.g., summer blockbusters, holiday releases).

month_dt → numeric (for calculations).

month → could later be turned into month names for plots.

(Small note: you might want dt.month_name() if you prefer full names like “July”)

# Step 4:
Drop raw release_date

In [25]:
tn_movie_budgets = tn_movie_budgets.drop(columns=['release_date'])


# Why?

We’ve extracted all useful parts (year + month).

Dropping avoids duplication and keeps dataframe cleaner.

# Step 5
Inspect

In [26]:
print(tn_movie_budgets[['movie','release_year','month_dt','month']].head())
print(tmdb_movies[['title','release_year']].head())


                                         movie  release_year  month_dt  month
0                                       Avatar          2009        12     12
1  Pirates of the Caribbean: On Stranger Tides          2011         5      5
2                                 Dark Phoenix          2019         6      6
3                      Avengers: Age of Ultron          2015         5      5
4            Star Wars Ep. VIII: The Last Jedi          2017        12     12
                                          title  release_year
0  Harry Potter and the Deathly Hallows: Part 1          2010
1                      How to Train Your Dragon          2010
2                                    Iron Man 2          2010
3                                     Toy Story          1995
4                                     Inception          2010


# Now you’re cleaning up the financial columns from The Numbers so they’re ready for calculations and plots. 

# Step 1: Identify the money columns

In [27]:
cols = ['production_budget', 'domestic_gross', 'worldwide_gross']


# Why?
These are stored as strings with $ and commas (e.g.,       "$100,000,000").
We can’t do math or plots with strings → must convert to numbers.

# Step 2: Remove $ and ,

In [28]:
tn_movie_budgets[cols] = tn_movie_budgets[cols].replace('[\$,]', '', regex=True)


# Why?

[\$,] means: match dollar signs $ or commas ,.

.replace(..., regex=True) strips them out → "100000000".

# Step 3: Convert to integers

In [29]:
tn_movie_budgets[cols] = tn_movie_budgets[cols].astype('int64')


# Why?

Converts cleaned strings into integers so we can:

Calculate profits/losses

Plot histograms, scatterplots

Run regressions

# Step 4 Inspect the result

In [31]:
print(tn_movie_budgets[cols].dtypes)   # confirm int64
tn_movie_budgets[cols].describe()      # check ranges, averages, etc.
tn_movie_budgets.head(3)               # preview cleaned values


production_budget    int64
domestic_gross       int64
worldwide_gross      int64
dtype: object


,id,movie,production_budget,domestic_gross,worldwide_gross,release_year,month_dt,month
0,1,Avatar,425000000,760507625,2776345279,2009,12,12
1,2,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011,5,5
2,3,Dark Phoenix,350000000,42762350,149762350,2019,6,6




describe() shows if values are realistic (e.g., budgets in millions, not billions).

# Standardizing titles across all datasets to improve your merge success rate

# Step 1: Apply .str.title() to titles

In [33]:
runtime_df['primary_title'] = runtime_df['primary_title'].str.title()
tn_movie_budgets['movie'] = tn_movie_budgets['movie'].str.title()
bom_movie_gross['title'] = bom_movie_gross['title'].str.title()
tmdb_movies['title'] = tmdb_movies['title'].str.title()


# Why?

In different datasets, titles may appear as "avatar", "Avatar", or "AVATAR".

.str.title() converts them all to "Avatar" → making matches more consistent when merging.

# Step 2: Inspect for consistency

In [35]:
print(runtime_df['primary_title'].head(5))
print(tn_movie_budgets['movie'].head(5))
print(bom_movie_gross['title'].head(5))
print(tmdb_movies['title'].head(5))


0                          Sunghursh
1    One Day Before The Rainy Season
2         The Other Side Of The Wind
4           The Wandering Soap Opera
Name: primary_title, dtype: object
0                                         Avatar
1    Pirates Of The Caribbean: On Stranger Tides
2                                   Dark Phoenix
3                        Avengers: Age Of Ultron
4              Star Wars Ep. Viii: The Last Jedi
Name: movie, dtype: object
0                                    Toy Story 3
1                     Alice In Wonderland (2010)
2    Harry Potter And The Deathly Hallows Part 1
3                                      Inception
4                            Shrek Forever After
Name: title, dtype: object
0    Harry Potter And The Deathly Hallows: Part 1
1                        How To Train Your Dragon
2                                      Iron Man 2
3                                       Toy Story
4                                       Inception
Name: title, dtype: obj

# Now you’re adding profit margin columns so you can analyze which movies actually made money relative to their costs.(tn_movie_budgets)

# Step 1: Domestic profit margin

In [40]:
tn_movie_budgets['dom_profit_margin'] = (
    (tn_movie_budgets['domestic_gross'] - tn_movie_budgets['production_budget'])
    / tn_movie_budgets['domestic_gross']
) * 100


Formula:

Profit Margin
=
Revenue
−
Cost
Revenue
×
100
Profit Margin=
Revenue
Revenue−Cost
	​

×100

Tells you what % of revenue was actual profit from U.S. box office only.

# Step 2: Worldwide profit margin

In [41]:
tn_movie_budgets['ww_profit_margin'] = (
    (tn_movie_budgets['worldwide_gross'] - tn_movie_budgets['production_budget'])
    / tn_movie_budgets['worldwide_gross']
) * 100



Same idea, but using global revenue.

Helps you see if movies depended more on domestic vs international markets for profitability.

# Step 3: Inspect results

In [42]:
tn_movie_budgets[['movie','production_budget','domestic_gross','worldwide_gross','dom_profit_margin','ww_profit_margin']].head(10)


,movie,production_budget,domestic_gross,worldwide_gross,dom_profit_margin,ww_profit_margin
0,Avatar,425000000,760507625,2776345279,44.116274,84.692106
1,Pirates Of The Caribbean: On Stranger Tides,410600000,241063875,1045663875,-70.328300,60.733080
2,Dark Phoenix,350000000,42762350,149762350,-718.477001,-133.703598
3,Avengers: Age Of Ultron,330600000,459005868,1403013963,27.974777,76.436443
4,Star Wars Ep. Viii: The Last Jedi,317000000,620181382,1316721747,48.885921,75.925058
5,Star Wars Ep. Vii: The Force Awakens,306000000,936662225,2053311220,67.330806,85.097242
6,Avengers: Infinity War,300000000,678815482,2048134200,55.805369,85.352522
7,Pirates Of The Caribbean: At WorldâS End,300000000,309420425,963420425,3.044539,68.860947
8,Justice League,300000000,229024295,655945209,-30.990470,54.264473
9,Spectre,300000000,200074175,879620923,-49.944389,65.894399


# This structure is like we did for profit margins, but now for profit amount and ROI — and using your dataset (tn_movie_budgets).

# Step 4: Worldwide profit amount

In [43]:
tn_movie_budgets['world_wide_profit_amount'] = (
    tn_movie_budgets['worldwide_gross'] - tn_movie_budgets['production_budget']
)


# Why?

This gives you the absolute dollar profit (or loss) a movie made globally.

Unlike margins, this shows the real money gained.

Example: If budget = $100M, worldwide gross = $250M →
Profit = $150M.

# Step 5: Return on Investment (ROI)

In [44]:
tn_movie_budgets['ROI_perc'] = (
    tn_movie_budgets['world_wide_profit_amount'] / tn_movie_budgets['production_budget']
) * 100


# Why
ROI tells you how efficiently money was used.

Formula:

𝑅
𝑂
𝐼
=
Net Profit
Budget
×
100
ROI=
Budget
Net Profit
	​

×100

A blockbuster making $200M profit on a $200M budget → ROI = 100%.

But a small film making $20M profit on $5M budget → ROI = 400%.

So ROI highlights hidden winners among low-budget films.

# Step 6:Inspect results

In [45]:
tn_movie_budgets[['movie','production_budget','worldwide_gross',
                  'world_wide_profit_amount','ROI_perc']].head(10)


,movie,production_budget,worldwide_gross,world_wide_profit_amount,ROI_perc
0,Avatar,425000000,2776345279,2351345279,553.257713
1,Pirates Of The Caribbean: On Stranger Tides,410600000,1045663875,635063875,154.667286
2,Dark Phoenix,350000000,149762350,-200237650,-57.210757
3,Avengers: Age Of Ultron,330600000,1403013963,1072413963,324.384139
4,Star Wars Ep. Viii: The Last Jedi,317000000,1316721747,999721747,315.369636
5,Star Wars Ep. Vii: The Force Awakens,306000000,2053311220,1747311220,571.016739
6,Avengers: Infinity War,300000000,2048134200,1748134200,582.711400
7,Pirates Of The Caribbean: At WorldâS End,300000000,963420425,663420425,221.140142
8,Justice League,300000000,655945209,355945209,118.648403
9,Spectre,300000000,879620923,579620923,193.206974


# Now filtering the dataset by year tn_movie_budgets

In [56]:
tn_movie_budgets= tn_movie_budgets[tn_movie_budgets['release_year'] > 2000]


# Why?

Older movies (before 2025) may not reflect today’s industry dynamics.

Budgets, marketing, and box office models changed drastically in the 2025s (e.g., streaming, globalization).



# Shifting into release month analysis. Since we are using tn_movie_budgets instead of numbers_df, let’s rewrite and break it down:

# Step 8: Group by release month and calculate medians

In [59]:
# Group movies by release month and take the median of numeric columns
month_df = tn_movie_budgets.groupby('month').median()

# Reset index so 'month' becomes a column again
month_df = month_df.reset_index()

# Sort by month number (1–12)
month_df = month_df.sort_values('month')

# Add month names
month_dict = {
    1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr',
    5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug',
    9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
}
month_df['month_name'] = month_df['month'].map(month_dict)


# Why?

Grouping by month lets you see if certain months tend to produce higher profits/ROI.

Using the median reduces the impact of extreme outliers (e.g., Avengers making billions).

Sorting ensures the months are in calendar order.

Adding names (Jan, Feb, etc.) makes plots readable.